In [46]:
import torch
from copy import deepcopy
from tqdm import tqdm
import os
from transformers import RobertaTokenizerFast

In [47]:
def mlm(tensor):
    # mask_arr = (torch.rand(tensor.shape) < 0.15) * (tensor != 0) * (tensor != 1) * (tensor != 2)
    rand = torch.rand(tensor.shape)
    # mask_arr = (rand < 0.15) * (tensor != 0) * (tensor != 1) * (tensor != 2)
    mask_arr = (rand < 0.15) * (tensor > 2)
    for i in range(tensor.shape[0]):
        selection = torch.flatten(mask_arr[i].nonzero()).tolist()
        tensor[i, selection] = 4
    return tensor

In [49]:
avar_tokenizer = RobertaTokenizerFast.from_pretrained('tokenizer_model_50')

In [51]:
# avar_tokenizer('хӏалтӏиги')

In [52]:
path = r'C:\m092\avar_bert\raw_corpora'
paths = [path + '\\' + fn for fn in os.listdir(path)]

In [53]:
input_ids = []
mask = []
labels = []

In [54]:
for path in tqdm(paths):
    with open(path, 'r', encoding="utf-8") as file:
        lines = file.read().split('\n')
    sample = avar_tokenizer(lines, max_length=512, padding='max_length', truncation=True, return_tensors='pt') # truncation=true обрезать слишком длинные строки
    labels.append(sample.input_ids)
    mask.append(sample.attention_mask)
    input_ids.append(mlm(sample.input_ids.detach().clone())) # .detach().clone() — как deepcopy мы не хотим портить настоящий список с id, т. к. он нужен в labels

100%|████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:14<00:00,  2.09s/it]


In [55]:
input_ids = torch.cat(input_ids)
mask = torch.cat(mask)
labels = torch.cat(labels)

In [58]:
torch.save(labels, 'av_labels.pt') # можно схранить
torch.save(mask, 'av_mask.pt') # можно схранить
torch.save(input_ids, 'av_input_ids.pt') # можно схранить

In [59]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        # store encodings internally
        self.encodings = encodings

    def __len__(self):
        # return the number of samples
        return self.encodings['input_ids'].shape[0]

    def __getitem__(self, i):
        # return dictionary of input_ids, attention_mask, and labels for index i
        return {key: tensor[i] for key, tensor in self.encodings.items()}

In [60]:
dataset = Dataset({'input_ids': input_ids, 'attention_mask': mask, 'labels': labels})

In [61]:
loader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True) # ???

In [45]:
# loader.dataset

In [62]:
from transformers import RobertaConfig
from transformers import RobertaForMaskedLM

config = RobertaConfig(
    vocab_size=50265,  # we align this to the tokenizer vocab_size
    max_position_embeddings=514,
    hidden_size=768,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1
)
model = RobertaForMaskedLM(config)

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

#### 1.4

In [24]:
for path in tqdm(paths):
    with open(path, 'r', encoding="utf-8") as file:
        lines = file.read().split('\n')
    sample = avar_tokenizer(lines, max_length=512, padding='max_length', truncation=True, return_tensors='pt') # truncation=true обрезать слишком длинные строки
    labels.append(sample.input_ids)
    mask.append(sample.attention_mask)
    input_ids.append(mlm(sample.input_ids.detach().clone())) # .detach().clone() — как deepcopy || мы не хотим портить настоящий список с id, т. к. он нужен в labels

100%|████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:14<00:00,  2.11s/it]


In [21]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        # store encodings internally
        self.encodings = encodings

    def __len__(self):
        # return the number of samples
        return self.encodings['input_ids'].shape[0]

    def __getitem__(self, i):
        # return dictionary of input_ids, attention_mask, and labels for index i
        return {key: tensor[i] for key, tensor in self.encodings.items()}

In [20]:
dataset = Dataset({'input_ids': input_ids, 'attention_mask': mask, 'labels': labels})

In [19]:
loader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True)

AttributeError: 'list' object has no attribute 'shape'